In [ ]:
# example configs

yaml_file = {'key_1' : 'value_1',
             'key_2' : {'subkey_1' : 'value_1',
                        'subkey_2' : 'value_2'}}

txt_file = """some text

              more text"""

## inputs

# solution id 

# path to conf
# list of files

## prepare inputs for tables
### read in files

## > solution_parameter_set table
# assamble parameter_set_id
# set deployment status to "STAGING"
# set insertion date 
# -> add

## > parameter set
# split parameters into individual ones based on file type
# hash each parameter to create parameter_id
# record paramerter type, name, file name, file type
# depending on value of the paramter, split it into attributes
# hash attributes and give then ids

## populate tables

# check with id if parameter set exists and if not add
# check with id if parameter exists and if not add (if yes assign existing to parameter set)
# check with id if attribute of parameter exists if not add (if yes assign to paramter)


## persistance

## > add solution description if not present
# add/change solution name (or pull solution name if not provided)
# add/change deployment date (pull if present or pick todays deployment date if not provided)
# edit deprication date
# edit maintainers

## > translate tables into database connector format

# upload parameter set
# pull parameter set
# update parameter set





In [8]:
import attr
import attrs
import hashlib
from datetime import datetime
from mocker_db import MockerDB
import logging

# connection handler
@attr.s
class DatabaseConnector:

    local_db_handler = attr.ib(default=MockerDB)

    logger = attr.ib(default=None)
    logger_name = attr.ib(default='Database Connector')
    loggerLvl = attr.ib(default=logging.INFO)
    logger_format = attr.ib(default=None)


    def __attrs_post_init__(self):
        self._initialize_logger()

    def _initialize_logger(self):

        """
        Initialize a logger for the class instance based on the specified logging level and logger name.
        """

        if self.logger is None:
            logging.basicConfig(level=self.loggerLvl, format=self.logger_format)
            logger = logging.getLogger(self.logger_name)
            logger.setLevel(self.loggerLvl)

            self.logger = logger

    def _initialize_mocker(self):

        self.local_db_handler = self.local_db_handler(file_path = "./parameterframe",
                                                        persist = False
                                                    )

    def add_entry(self, table_name: str, entry) -> bool:
        try:


            # Check if entry is an instance of an attrs class and serialize accordingly
            if hasattr(entry, '__attrs_attrs__'):
                entry_dict = {a.name: getattr(entry, a.name) for a in attr.fields(entry.__class__) \
                    if a.name not in ['__tablename__', 'database_connector']}
            else:
                # Assume entry is already a dictionary
                entry_dict = entry

            self.local_db_handler.data[table_name].append(entry_dict)
        except Exception as e:
            self.logger.error(f"Failed to add entry: {e}")
            return False

        return True




# table handlers
## solution_description
@attr.s
class SolutionDescription():
    __tablename__ = 'solution_description'

    database_connector = attr.ib()

    solution_id = attr.ib(init = False, type=str)
    solution_name = attr.ib(init = False, type=str)
    deployment_date = attr.ib(init = False, type=datetime)
    deprication_date = attr.ib(init = False, type=datetime)
    maintainers = attr.ib(init = False, type = list)

    def _prepare_maintainers(self, maintainers : list) -> str:

        # turn list into formated str
        return ', '.join(name.capitalize() for name in maintainers)

    def _make_solution_id(self,
                          solution_name : str,
                          deployment_date : datetime,
                          maintainers : str) -> str:

        input_string = solution_name + str(deployment_date) + str(maintainers)

        return hashlib.sha256(input_string.encode()).hexdigest()

    def add_solution(self,
                    solution_name : str,
                    deployment_date : datetime,
                    maintainers : list,
                    deprication_date  : datetime = None,
                    solution_id : str = None) -> bool:

        """
        Add solution description.
        """

        try:

            self.solution_name = solution_name
            self.deployment_date = deployment_date
            self.maintainers = self._prepare_maintainers(maintainers)
            if deprication_date is None:
                self.deprication_date = None
            if solution_id is None:
                self.solution_id = self._make_solution_id(
                            solution_name = solution_name,
                            deployment_date = deployment_date,
                            maintainers = maintainers)

            self.database_connector.add_entry(table_name = self.__tablename__,
                                              entry = self)
        except:
            return False

        return True

    def update_solution_parameter(self,
                                solution_id : str,
                                solution_name : str = None,
                                deployment_date : datetime = None,
                                deprication_date  : datetime = None,
                                maintainers : list = None) -> bool:

        """
        Update one or multiple solution description parameters.
        """

        try:

            self.solution_id = solution_id

            if solution_name is not None:
                self.solution_name = solution_name
            if deployment_date is not None:
                self.deployment_date = deployment_date
            if deprication_date is not None:
                self.deprication_date = deprication_date
            if maintainers is not None:
                self.maintainers = self._prepare_maintainers(maintainers)

        except:
            return False

        return True

    def remove_solution(self,
                        solution_id : str,
                        solution_name : str = None,
                        deployment_date : datetime = None,
                        deprication_date  : datetime = None,
                        maintainers : list = None) -> bool:

        """
        Update one or multiple solution description parameters.
        """

        try:

            self.solution_id = solution_id

            if solution_name is not None:
                self.solution_name = solution_name
            if deployment_date is not None:
                self.deployment_date = deployment_date
            if deprication_date is not None:
                self.deprication_date = deprication_date
            if maintainers is not None:
                self.maintainers = self._prepare_maintainers(maintainers)

        except:
            return False

        return True


In [16]:
md = MockerDB(
    file_path = "./parameterframe",
    persist = False
)

In [21]:
md.search_database(query = '',
                   #filter_criteria = {'table_name' : 'table_name'},
                   perform_similarity_search = False,
                   return_keys_list=['solution_name'])

[{'solution_name': 'test'}]

In [19]:
md.insert_values(values_dict_list=[{'solution_id': 'cf72e48f0d39d3a3a2b79476848c17d65055926b46515c507e9c25bb2b92a056',
   'solution_name': 'test',
   'deployment_date': '2000-01-01',
   'deprication_date': None,
   'maintainers': 'Me, And me'}], var_for_embedding_name='text',embed = False)

In [9]:
dc = DatabaseConnector()
sd = SolutionDescription(database_connector=dc)

In [10]:
sd.add_solution(solution_name='test',
                deployment_date='2000-01-01',
                maintainers=['me','and me'])

True

In [12]:
sd.database_connector.data

{'solution_description': [{'solution_id': 'cf72e48f0d39d3a3a2b79476848c17d65055926b46515c507e9c25bb2b92a056',
   'solution_name': 'test',
   'deployment_date': '2000-01-01',
   'deprication_date': None,
   'maintainers': 'Me, And me'}]}

### solution_parameter table 
solution id, parameter_set_id, deployment status, insertion datetime 

### parameter_set_parameter_id table
parameter_set_id, parameter_id

### parameter_file table
parameter_id, file_name, file_type

### solution_id table 
solution id, deployment_date, deprication_date, solution name, maintainers

### parameter_id table
solution_id, param_id, param_type [scalar | nested], param_name_id, param_value

### solution_parameter table
solution_id, param_name_id, attribute_name_id, param_name, attribute_name

### paramter_attribute table
parameter_id, attribute_id, attribute_name_id, attribute_value

In [ ]:
class SolutionParameter:
    def __init__(self, solution_id, parameter_set_id, deployment_status, insertion_datetime):
        self.solution_id = solution_id
        self.parameter_set_id = parameter_set_id
        self.deployment_status = deployment_status
        self.insertion_datetime = insertion_datetime

class ParameterSetParameterId:
    def __init__(self, parameter_set_id, parameter_id):
        self.parameter_set_id = parameter_set_id
        self.parameter_id = parameter_id

class ParameterFile:

    """
    The table is designed to manage the association between parameters and their corresponding source files.
    """

    def __init__(self, parameter_id, file_name, file_type):
        self.parameter_id = parameter_id
        self.file_name = file_name
        self.file_type = file_type

    def add_file_association(self):
        # Logic to add this file association to the database
        pass

    @staticmethod
    def get_files_by_parameter_id(parameter_id):
        # Logic to retrieve file associations for a specific parameter
        pass

    def update_file_association(self, new_file_details):
        # Logic to update this file association's details
        pass

    @staticmethod
    def delete_file_association(parameter_id):
        # Logic to delete a file association by parameter ID
        pass

class SolutionId:

    """
    The table is aimed at managing various solutions along with their deployment and deprecation details.
    """

    def __init__(self, solution_id, deployment_date, deprecation_date, solution_name, maintainers):
        self.solution_id = solution_id
        self.deployment_date = deployment_date
        self.deprecation_date = deprecation_date
        self.solution_name = solution_name
        self.maintainers = maintainers

    def create_solution(self):
        # Logic to register this solution in the database
        pass

    @staticmethod
    def get_solution_by_id(solution_id):
        # Logic to retrieve a solution by its ID
        pass

    def update_solution(self, new_values):
        # Logic to update this solution's details
        pass

    @staticmethod
    def delete_solution(solution_id):
        # Logic to delete a solution by its ID
        pass

class ParameterId:

    """
    The table designed to manage parameters including their types, values, and associations with specific solutions,
    the corresponding class should include a variety of methods to effectively handle these aspects.
    """

    def __init__(self, solution_id, param_id, param_type, param_name_id, param_value):
        self.solution_id = solution_id
        self.param_id = param_id
        self.param_type = param_type
        self.param_name_id = param_name_id
        self.param_value = param_value

    def add_parameter(self):
        # Logic to add this parameter to the database
        pass

    @staticmethod
    def get_parameter_by_id(param_id):
        # Logic to retrieve a parameter by its ID
        pass

    def update_parameter(self, new_values):
        # Logic to update this parameter's details
        pass

    @staticmethod
    def delete_parameter(param_id):
        # Logic to delete a parameter by its ID
        pass

class SolutionParameterDetails:

    """
    The table designed to manage the relationship between solutions and their parameters along with additional attributes.
    As a class it should include methods for managing these relationships and attributes.
    """

    def __init__(self, solution_id, param_name_id, attribute_name_id, param_name, attribute_name):
        self.solution_id = solution_id
        self.param_name_id = param_name_id
        self.attribute_name_id = attribute_name_id
        self.param_name = param_name
        self.attribute_name = attribute_name

    def add_solution_parameter(self):
        # Logic to add this entry to the database
        pass

    @staticmethod
    def get_parameters_by_solution(solution_id):
        # Logic to retrieve parameters for a specific solution
        pass

    def update_solution_parameter(self, new_values):
        # Logic to update this entry's details
        pass

    @staticmethod
    def delete_solution_parameter(solution_id, param_name_id):
        # Logic to delete a specific solution-parameter relationship
        pass

class ParameterAttribute:

    """
    The table involves handling different types of parameters associated with solutions,
    extending them with additional attributes.
    """


    def __init__(self, parameter_id, attribute_id, attribute_name_id, attribute_value):
        self.parameter_id = parameter_id
        self.attribute_id = attribute_id
        self.attribute_name_id = attribute_name_id
        self.attribute_value = attribute_value

    def add_attribute(self):
        # Logic to add this attribute to the database
        pass

    @staticmethod
    def get_attribute_by_id(attribute_id):
        # Logic to get an attribute by ID
        pass

    def update_attribute_value(self, new_value):
        # Logic to update the attribute's value
        self.attribute_value = new_value
        # Update in database
        pass

    @staticmethod
    def delete_attribute(attribute_id):
        # Logic to delete an attribute by ID
        pass


In [3]:
import os
import yaml
import csv
import attr
import attrs
from datetime import datetime
from uuid import uuid4

@attr.s
class ParameterFrame:

    sync_path = attr.ib(default=None)
    solution_id = attr.ib(default=None)
    #solution_name = attr.ib(default=None)

    overwrite = attr.ib(default=True)



    def _generate_unique_id(self):
        return str(uuid4())

    def _process_yaml_file(self, file_path):
        with open(file_path, 'r') as file:
            params = yaml.safe_load(file)
            self._add_parameters_from_dict(params)

    def _process_txt_file(self, file_path):
        # Implement the logic to read and parse TXT files
        pass

    def _add_parameters_from_dict(self, params, parent_id=None):
        for key, value in params.items():
            param_id = self._generate_unique_id()
            param_type = 'nested' if isinstance(value, dict) else 'scalar'
            self.parameter.append({
                "solution_id": parent_id,
                "param_id": param_id,
                "param_type": param_type,
                "param_name_id": key,
                "param_value": value if param_type == 'scalar' else None
            })

            if param_type == 'nested':
                self._add_parameters_from_dict(value, param_id)

    def create_parameter_set(self):
        pass

    def export_parameters_to_file(self, parameter_set_id, file_format):
        # Implement logic to export parameters to a specified file format
        pass

    def _save_table_to_csv(self, table, file_name):
        # Same as previous implementation
        pass

    def _check_if_parameter_set_exists(self):
        pass

    def _establish_connection(self):
        pass

    def add_parameters(self):
        for file_name in os.listdir(self.folder_path):
            full_path = os.path.join(self.folder_path, file_name)
            if file_name.endswith('.yaml'):
                self._process_yaml_file(full_path)
            elif file_name.endswith('.txt'):
                self._process_txt_file(full_path)



    def log_solution(self, solution_name, maintainers, deployment_date=None, deprecation_date=None):
        solution_id = self._generate_unique_id()
        self.solution.append({
            "solution_id": solution_id,
            "deployment_date": deployment_date or datetime.now(),
            "deprecation_date": deprecation_date,
            "solution_name": solution_name,
            "maintainers": maintainers
        })
        # Generate parameter_set_id and link to solution_id
        parameter_set_id = self._generate_unique_id()
        # Linking process (not fully implemented here)

    def change_deployment_status(self):
        pass

    def display_parameters(self):
        pass

    def push_parameter(self):
        pass

    def commit_parameters(self):
        pass

    def pull_parameters(self):
        pass

    def switch_parameters(self):
        pass


